In [1]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *

base_path = "/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/ParlaMintCase"

In [2]:
df = data_loader.full('pl','preprocessed')
df['date'] = [utils.find_date(x) for x in df['id']]
df = utils.add_metadata(df,'pl')
df['text'] = [re.sub(r'[^A-Za-z0-9 ]+', '', str(s)) for s in df['text']]

In [3]:
tf = pd.read_csv('/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/ParlaMintCase/resources/keywords-corona-translation.csv')
tf_corona = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}

In [4]:
df = df[(df['text'].str.contains("|".join(tf_corona['pl'].values())))].reset_index(drop=True)

In [5]:
tfidf_obj,docterms = tfidf.get_docterms(df,'text')
tfidf_df = tfidf.get_topterms(tfidf_obj,docterms,df,'speaker_party')

In [10]:
for c in tfidf_df.columns:
    print(c,"\t\t"," ".join(tfidf_df[c][:15]))

PiS terms 		 ustawy oraz pani komisji nr prosz przez zwizku tych od bo dla bdzie wirusa porzdku
_ terms 		 eby od dla chodzi przez jeeli ktry bdzie czyli tutaj ju tych rwnie pani dzisiaj
Kukiz'15 terms 		 konkretne pose rzecz bardzo oklaski prosz pastwa tych ktrzy chyba polskich ile raz prosz powiedzie pytanie dzikuj bardzo czas
Teraz! terms 		 klski tego typu kryzysu typu jakim nasze tak naprawd pastwo prosz pastwa albo czas naprawd ustawa polsce kiedy
Lewica terms 		 bdzie ma pan koronawirusa pani od wysoka izbo izbo wysoka przez dla bo rzd ustawy pytanie
Niezrzeszeni terms 		 ja pan mwi dzisiaj eby bo tysicy pastwo poprawki pytanie bdzie tych mam ma jeeli
KP-PSL terms 		 dla dzisiaj pani bo pytanie tylko pana wysoka dzikuj bardzo wysoka izbo izbo bdzie ktry ustawy przez
Przywrócić Prawo terms 		 posowie byli sprawiedliwoci prosz pastwa dalej panu spraw posa mnie razie ja wyborw pana mia pastwa
KO;Niezrzeszeni terms 		 nawet rzecznika gdy mwi pan dzieci brak wicej rzecznik europie od

In [14]:
pd.DataFrame(docterms)

,0
0,"(0, 39)\t0.06993359678268656\n (0, 154)\t0...."
1,"(0, 501)\t0.07516733740786703\n (0, 377)\t0..."
2,"(0, 371)\t0.0874127220673454\n (0, 865)\t0...."
3,"(0, 418)\t0.09629925051305618\n (0, 789)\t0..."
4,"(0, 240)\t0.058820363563666246\n (0, 29)\t0..."
...,...
1615,"(0, 54)\t0.08176283562383697\n (0, 265)\t0...."
1616,"(0, 273)\t0.20596874662118322\n (0, 523)\t0..."
1617,"(0, 812)\t0.07218691379709549\n (0, 272)\t0..."
1618,"(0, 811)\t0.07750612824356712\n (0, 461)\t0..."
